# Step 1: Import modules and build classes

In [ ]:
# Basic imports (yeah it's not much)
import numpy as np
import os

The PDEs are solved by applying certain boundary conditions which indicate how the fluid will behave at the boundaries of the domain. For example, fluid flowing through a pipe will have a wall with zero fluid velocity and an entry as well as exit with some specified flow velocity.

Mathematically, boundary conditions can be expressed in two forms — Dirichlet and Neumann boundaries. The former specifies a value of the dependent variable at the boundary whereas the latter specifies a value for the derivative of the dependent variable at the boundary.

In [ ]:
# Therefore, we make a Boundary class that has two properties — type and value.
class Boundary:
    def __init__(self, boundary_type, boundary_value):
        self.DefineBoundary(boundary_type, boundary_value)
        
    def DefineBoundary(self,boundary_type, boundary_value):
        self.type=boundary_type
        self.value=boundary_value

The domain enclosed by the boundary (like the inside of a pipe) is represented using a 2D mesh or grid and the values of dependent variables are calculated at the center of boxes in the grid (for pressure) or at the faces of the boxes (for velocities). 

This is referred to as a staggered grid approach. To represent the mesh, we create a class called Space. 
The method CreateMesh creates a matrix of given size for the dependent variables and the SetDeltas method calculates the values of the differential lengths based on the specified length and breadth of the domain.

In [ ]:
class Space:
    def __init__(self):
        pass
    
    def CreateMesh(self, rowpts, colpts):
        #Domain gridpoints
        self.rowpts = rowpts
        self.colpts = colpts
        
        #Velocity matrices
        self.u = np.zeros((self.rowpts+2, self.colpts + 2))
        self.v = np.zeros((self.rowpts+2, self.colpts + 2))
        
        self.u_star = np.zeros((self.rowpts + 2, self.colpts + 2))
        self.v_star = np.zeros((self.rowpts + 2, self.colpts + 2))
        
        self.u_next = np.zeros((self.rowpts + 2, self.colpts + 2))
        self.v_next = np.zeros((self.rowpts + 2, self.colpts + 2))
        
        self.u_c = np.zeros((self.rowpts, self.colpts))
        self.v_c = np.zeros((self.rowpts, self.colpts))
        
        #Pressure matrices
        self.p = np.zeros((self.rowpts + 2, self.colpts + 2))
        self.p_c = np.zeros((self.rowpts, self.colpts))

        #Set default source term
        self.SetSourceTerm()        
        
    def SetDeltas(self,breadth, length):
        self.dx = length / (self.colpts - 1)
        self.dy = breadth / (self.rowpts - 1)
        
    def SetInitialU(self, U):
        self.u=U*self.u
        
    def SetInitialV(self, V):
        self.v=V*self.v
        
    def SetInitialP(self, P):
        self.p=P*self.p
        
    def SetSourceTerm(self, S_x = 0, S_y = 0):
        self.S_x = S_x
        self.S_y = S_y

Lastly, we create a class Fluid to represent the properties of the fluid — like density (rho) and viscosity (mu).

In [ ]:
class Fluid:
    def __init__(self, rho, mu):
        self.SetFluidProperties(rho, mu)
    
    def SetFluidProperties(self, rho, mu):
        self.rho = rho
        self.mu = mu

# Step 2: Write Functions to implement the Finite Difference Method

In [ ]:
#Note: The arguments to the function are all objects of our defined classes
#Set boundary conditions for horizontal velocity
def SetUBoundary(space, left ,right ,top ,bottom):
    if(left.type == "D"):
        space.u[:,0] = left.value
    elif(left.type == "N"):
        space.u[:,0] = -left.value * space.dx + space.u[:,1]
    
    if(right.type == "D"):
        space.u[:,-1] = right.value
    elif(right.type == "N"):
        space.u[:,-1] = right.value * space.dx + space.u[:,-2]
        
    if(top.type == "D"):
        space.u[-1,:] = 2 * top.value - space.u[-2,:]
    elif(top.type=="N"):
        space.u[-1,:] = -top.value * space.dy + space.u[-2,:]
     
    if(bottom.type == "D"):
        space.u[0,:] = 2 * bottom.value - space.u[1,:]
    elif(bottom.type == "N"):
        space.u[0,:] = bottom.value * space.dy + space.u[1,:]

In [ ]:

#Set boundary conditions for vertical velocity
def SetVBoundary(space,left,right,top,bottom):
    if(left.type=="D"):
        space.v[:,0]=2*left.value-space.v[:,1]
    elif(left.type=="N"):
        space.v[:,0]=-left.value*space.dx+space.v[:,1]
    
    if(right.type=="D"):
        space.v[:,-1]=2*right.value-space.v[:,-2]
    elif(right.type=="N"):
        space.v[:,-1]=right.value*space.dx+space.v[:,-2]
        
    if(top.type=="D"):
        space.v[-1,:]=top.value
    elif(top.type=="N"):
        space.v[-1,:]=-top.value*space.dy+space.v[-2,:]
     
    if(bottom.type=="D"):
        space.v[0,:]=bottom.value
    elif(bottom.type=="N"):
        space.v[0,:]=bottom.value*space.dy+space.v[1,:]

In [ ]:
#Set boundary conditions for pressure
def SetPBoundary(space,left,right,top,bottom):
    if(left.type=="D"):
        space.p[:,0]=left.value
    elif(left.type=="N"):
        space.p[:,0]=-left.value*space.dx+space.p[:,1]
    
    if(right.type=="D"):
        space.p[1,-1]=right.value
    elif(right.type=="N"):
        space.p[:,-1]=right.value*space.dx+space.p[:,-2]
        
    if(top.type=="D"):
        space.p[-1,:]=top.value
    elif(top.type=="N"):
        space.p[-1,:]=-top.value*space.dy+space.p[-2,:]
     
    if(bottom.type=="D"):
        space.p[0,:]=bottom.value
    elif(bottom.type=="N"):
        space.p[0,:]=bottom.value*space.dy+space.p[1,:]